<a href="https://colab.research.google.com/github/Shahsawar51/MY_DATA_SCIENCE_JOURNEY/blob/main/wk24_ss52_Mini_batch_gradient_decent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
pip install --upgrade scikit-learn


In [25]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import SGDRegressor

In [26]:
# Load the diabetes dataset
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

In [27]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [29]:
# Initialize SGDRegressor
sgd_regressor = SGDRegressor(
    max_iter=500,                # Total epochs
    learning_rate='constant',    # Fixed learning rate
    eta0=0.0001,                 # Initial learning rate
    random_state=42             # For reproducibility
)

# Fit the model on scaled training data
sgd_regressor.fit(X_train_scaled, y_train)

# Make predictions
y_pred = sgd_regressor.predict(X_test_scaled)

# Evaluate the model
print(f'Intercept: {sgd_regressor.intercept_}')
print(f'Coefficients: {sgd_regressor.coef_}')
print(f'R² Score: {r2_score(y_test, y_pred)}')


Intercept: [153.74109845]
Coefficients: [  1.95120855 -11.41814927  26.31732965  16.58815808  -7.22495972
  -4.29746049  -8.95059172   7.67684916  20.96552165   2.63753176]
R² Score: 0.45541203739843916


In [30]:
class SGDRegressor:
    def __init__(self, learning_rate=0.01, epochs=100, batch_size=1):
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size

    def fit(self, X_train, y_train):
        np.random.seed(42)
        self.intercept_ = 0
        self.coef_ = np.zeros(X_train.shape[1])

        for epoch in range(self.epochs):
            indices = np.arange(X_train.shape[0])
            np.random.shuffle(indices)

            # Mini-batch training
            for i in range(0, X_train.shape[0], self.batch_size):
                batch_indices = indices[i:i + self.batch_size]
                X_batch = X_train[batch_indices]
                y_batch = y_train[batch_indices]

                y_pred = np.dot(X_batch, self.coef_) + self.intercept_

                # Compute gradients (batch mean)
                intercept_der = -2 * np.mean(y_batch - y_pred)
                coef_der = -2 * np.dot(X_batch.T, (y_batch - y_pred)) / self.batch_size

                # Update parameters
                self.intercept_ -= self.lr * intercept_der
                self.coef_ -= self.lr * coef_der

        # Final loss
        final_loss = np.mean((y_train - (np.dot(X_train, self.coef_) + self.intercept_)) ** 2)
        print(f'Final Loss: {final_loss}, Intercept: {self.intercept_}, Coefficients: {self.coef_}')

    def predict(self, X_test):
        return np.dot(X_test, self.coef_) + self.intercept_

In [42]:
# Initialize and train the custom SGDRegressor
custom_sgd = SGDRegressor(learning_rate=0.001, epochs=100, batch_size=5)
custom_sgd.fit(X_train_scaled, y_train)

# Make predictions
y_pred_custom = custom_sgd.predict(X_test_scaled)

print(f'R² Score: {r2_score(y_test, y_pred_custom)}')

Final Loss: 2892.9146996075197, Intercept: 153.69125135760908, Coefficients: [  1.91904614 -11.42251949  26.22746724  16.53677013  -8.01922034
  -3.83065054  -8.57727983   7.77217929  21.18062868   2.57797163]
R² Score: 0.45527464333622225
